In [ ]:
import pydicom
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
dcmImage = pydicom.dcmread("trial.dcm")
dcm_scan = dcmImage.pixel_array
print(np.shape(dcm_scan))

In [ ]:
# preprocess the data

In [ ]:
# use keras inception v3 to train on the time stamped image frames
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

In [ ]:
def create_model():
    base_model = InceptionV3(weights='imagenet', include_top=False)

    # average pooling layer
    x = GlobalAveragePooling2D()(base_model.output)
    # fully-connected layer
    x = Dense(1024, activation='relu')(x)
    # logistic layer
    predictions = Dense(len(data.classes), activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def create_generators():
    train_gen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        horizontal_flip=True,
        rotation_range=10.,
        width_shift_range=0.2,
        height_shift_range=0.2).flow_from_directory(
        './data/train/',
        target_size=(299, 299),
        batch_size=32,
        classes=data.classes,
        class_mode='categorical')

    validation_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
        './data/test/',
        target_size=(299, 299),
        batch_size=32,
        classes=data.classes,
        class_mode='categorical')

    return train_generator, validation_generator

def train_model(model, nb_epoch, generators, callbacks=[]):
    train_generator, validation_generator = generators
    model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        validation_data=validation_generator,
        validation_steps=10,
        epochs=nb_epoch,
        callbacks=callbacks)
    return model

In [ ]:
checkpointer = ModelCheckpoint(
    filepath='./data/checkpoints/inception.{epoch:03d}-{val_loss:.2f}.hdf5',
    verbose=1,
    save_best_only=True)

early_stopper = EarlyStopping(patience=10)

tensorboard = TensorBoard(log_dir='./data/logs/')

model = create_model()
generators = get_generators()

model = train_model(model, 10, generators)

In [ ]:
# general crossvalidation skeleton

from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold

tprs = []
aucs = []
i = 1

for train, test in cv.split(X, Y):
    model.fit(X[train], Y[train])
    Y_pred = model.predict(X[test])
    fpr, tprs, thresholds = roc_curve(Y[test], Y_pred)
    tpr.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw = 1, alpha = 0.3, label = 'ROC fold %d (AUC - %0.2f)' % (i, roc_auc))
    plt.legend()
    i+=1
    
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Random Chance', alpha=.8)

mean_fpr = np.linspace(0, 1, 100)
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = metrics.auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc))

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('PLS-DA ROC Curve with Crossvalidation')
plt.legend(loc="lower right")
plt.savefig('ROC Curve for PLS-DA with Crossvalidation-14comp.jpg')